In [1]:
import findspark
findspark.init('/usr/local/spark')

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import split , explode , count

In [3]:
spark = SparkSession.builder.appName("Pyspark Streaming Testing 2").getOrCreate()

In [4]:
socketDf = spark.readStream.format('socket')\
           .option('host','localhost')\
           .option('port','9999')\
           .load()

In [5]:
socketDf = socketDf.withColumnRenamed('_c0','Value') # Giving a Proper Column Name
splitDf = socketDf.select(split('Value'," ").alias('Value')) # Splittin the Value Column based on ' ' (empty space)
flattenedDF = splitDf.select(explode('Value').alias('Value')) # Exploding the Values in the Value Column
# Aggregating the Values based on their count
WordCountDf = flattenedDF.groupby('Value').agg(count('*').alias('count'))

In [6]:
results = WordCountDf.writeStream\
          .outputMode('update')\
          .format('memory')\
          .queryName('stm_wc')\
          .start()

In [7]:
#Only for this VM.
#Start your Hadoop Before this.
#Whenever there is an error 'connection refused' it will mostly be due to hadoop not being started.
#Before this Go to terminal and type 'nc -l 9999',then restart ther kernel and run all blocks of code.
#Pass the Data in Terminal --> Continuous Data
from IPython.display import display, clear_output
from time import sleep

while True:
    clear_output(wait=True)
    display(results.status)
    display(spark.sql('Select * From stm_wc').show())
    sleep(3)

{'isDataAvailable': False,
 'isTriggerActive': False,
 'message': 'Waiting for data to arrive'}

+--------+-----+
|   Value|count|
+--------+-----+
|    saif|    1|
|    hari|    2|
|   jason|    3|
|   jacob|    1|
|robinson|    2|
|  robbie|    1|
|  robbie|    2|
|        |    1|
| patrick|    1|
+--------+-----+



None

KeyboardInterrupt: 